In [1]:
import serial
import serial.tools.list_ports
import time
import matplotlib.pyplot as plt
from coincidence_counter_driver import CoincidenceCounter
from motor_driver import MotorDriver
from bi_photon_source import BiPhotonSource

In [ ]:
#pip install pyserial

In [ ]:
GROUP_NAME = 'Fr 1pm'

In [ ]:
ports = serial.tools.list_ports.comports()

for port, desc, hwid in sorted(ports):
        print(f"{port}:\t{desc} [{hwid}]")

In [ ]:
from IPython.display import clear_output
from matplotlib import pyplot as plt
import numpy as np
import collections
%matplotlib inline

def live_plot(data_dict: dict, x_axis, x_axis_label = '', title='', figsize=(7,5)):
    clear_output(wait=True)
    plt.figure(figsize=figsize)
    for label, data in data_dict.items():
        plt.plot(x_axis, data, label=label)
    plt.title(title)
    plt.grid(True)
    plt.xlabel(x_axis_label)
    plt.ylabel('counts')
    plt.legend(loc='center left') # the plot evolves to the right
    plt.show()

In [ ]:
motor = MotorDriver('COM18')

In [ ]:
coin = CoincidenceCounter('COM5')

In [ ]:
bi_photon_source = BiPhotonSource('COM4')

In [ ]:
motor.position = 10

In [ ]:
coin.ch1_delay = 0
coin.ch1_delay

In [ ]:
coin.dwell_time = 1.0
coin.dwell_time

In [ ]:
coin.coincidence_window = 3
coin.coincidence_window

In [ ]:
coin.query("DELA?")

In [ ]:
counts = coin.measure_channels()
print(f" ch1: {counts[0]}\n ch2: {counts[1]}\n coincidences: {counts[2]}")

In [ ]:
motor.position = 1
motor.position

In [ ]:
motor.position

# Experiment 3: Temporal sensitivity of bi-photons

In [ ]:
data_dict = {
    'ch1': [],
    'ch2': [],
    'coincidences': []
}
x_axis = []
for i in range(8):
    coin.ch1_delay = int(2*i)
    time.sleep(0.2)
    ch1, ch2, coincidences = coin.measure_channels()
    data_dict['ch1'].append(ch1)
    data_dict['ch2'].append(ch2)
    data_dict['coincidences'].append(coincidences)
    x_axis.append(2*i)
    live_plot(data_dict, x_axis,
              x_axis_label = "Delay of channel 1",
              title = 'Channel and coincidence counts over time')

# Hong-Ou-Mandel (HOM) experiment

In [ ]:
coin.measure_channels()

In [ ]:
import numpy as np

In [ ]:
hom_counts = {
    'ch1': [],
    'ch2': [],
}
hom_coincidences = {
    'coincidences': []
}
x_axis = []

coin.dwell_time = 1.0
coin.coincidence_window = 3
coin.ch1_delay = 0

step = 0.25
steps = np.arange(16, 28+step, step)
print(steps)
steps = np.flip(steps)
for i in steps:
    aim = i
    print(aim)
    motor.position = aim

    t0 = time.time()
    while abs(motor.position - aim) > 0.1:
        print(
            f"Current position is {motor.position:.2f} mm moving to {aim} mm",
            end  = '\r')
        if (time.time()-t0) > 2:
            motor.position = aim
            t0 = time.time()
        time.sleep(0.01)
    ch1, ch2, coincidences = coin.measure_channels()
    hom_counts['ch1'].append(ch1)
    hom_counts['ch2'].append(ch2)
    hom_coincidences['coincidences'].append(coincidences)
    x_axis.append(i)
    live_plot(hom_coincidences, x_axis,
              x_axis_label = "Motor position in mm",
              title = 'Coincidence counts vs motor position')

### SAVE THE DATA
# hom_data = hom_coincidences | hom_counts
# timestr = time.strftime("%Y%m%d-%H%M%S")
# with open(f'data/{GROUP_NAME}_exp4_HOM_{timestr}.txt', 'w') as f:
#     print(hom_data, file=f)

In [ ]:
motor.position = 20

In [ ]:
motor.position

In [ ]:
N=10
hom_counts = {
    'ch1': np.zeros(N),
    'ch2': np.zeros(N),
}
hom_coincidences = {
    'coincidences': np.zeros(N)
}
x_axis = np.arange(N)

coin.dwell_time = 1.0
coin.coincidence_window = 3
coin.ch1_delay = 0


        
for i in range (0, N):
    time.sleep(1)
    ch1, ch2, coincidences = coin.measure_channels()
    hom_counts['ch1'][i] = ch1
    hom_counts['ch2'][i] = ch2
    hom_coincidences['coincidences'][i] = coincidences
    live_plot(hom_coincidences, x_axis,
              x_axis_label = "test number",
              title = 'Coincidence counts vs motor position')


In [ ]:
maxx = max(hom_coincidences['coincidences'])
minn = min(hom_coincidences['coincidences'])
avg = np.average(hom_coincidences['coincidences'])
std = np.std(hom_coincidences['coincidences'])

print(maxx, minn, avg, std)

In [ ]:
maxx = max(hom_counts['ch1'])
minn = min(hom_counts['ch1'])
avg = np.average(hom_counts['ch1'])
std = np.std(hom_counts['ch1'])

print(maxx, minn, avg, std)

In [ ]:
maxx = max(hom_counts['ch2'])
minn = min(hom_counts['ch2'])
avg = np.average(hom_counts['ch2'])
std = np.std(hom_counts['ch2'])

print(maxx, minn, avg, std)

In [ ]:
### SAVE THE DATA
hom_data = hom_coincidences,hom_counts
timestr = time.strftime("%Y%m%d-%H%M%S")
with open(f'data/45000_exp4_HOM_{timestr}.txt', 'w') as f:
    print(hom_data, file=f)